In [1]:
import numpy as np
import pandas as pd
import json
import operator

def get_most_frquen_ans(answers):
    result = {}
    for i in range(10):
        result[answers[i]['answer']] = 1
    print(result)
    for i in range(10):
        result[answers[i]['answer']] +=1
    print('after :', result)
    return max(result.items(), key=operator.itemgetter(1))[0]

In [3]:
#测试下方法
json_obj = json.load(open("./test.json", "r"))
get_most_frquen_ans(json_obj['answers'])

{'oval': 1, 'semi circle': 1, 'curved': 1, 'double curve': 1, 'banana': 1, 'wavy': 1, 'twisting': 1}
after : {'oval': 2, 'semi circle': 2, 'curved': 5, 'double curve': 2, 'banana': 2, 'wavy': 2, 'twisting': 2}


'curved'

In [48]:
#读取图片的features
#使用标准coco的图片，从https://cs.stanford.edu/people/karpathy/deepimagesent/下可以下载到这些图片的features
#这样可以不用训练，直接拿到图片的feature
import scipy.io as io
#加载训练好的图片的feature到feature_struct中
#生成图片id与feature列标的对应数据 img_map
def generate_coco_image_feature(img_feats_file, img_ids_feats):
    feature_struct = io.loadmat(img_feats_file)
    img_vgg_features = feature_struct['feats']
    img_ids_feats_col = open(img_ids_feats).read().splitlines()
    img_map = {}
    for ids in img_ids_feats_col:
        ids_split = ids.split()
        img_map[ids_split[0]] = int(ids_split[1])
    return img_vgg_features, img_map

#根据图片id,取得相应的特征数据
def get_image_matrix(feature_struct, img_map, image_ids):
    rows = len(image_ids)
    img_matrix = np.zeros((rows, feature_struct.shape[0]))
    for i in range(rows):
        img_matrix[i,:] = feature_struct[:,img_map[image_ids[i]]]
    return img_matrix

In [95]:
# 处理文字
import gensim
from nltk.corpus import stopwords
from nltk import word_tokenize

stop_words = stopwords.words('english')
w2v_model = gensim.models.KeyedVectors.load_word2vec_format("./sentiment/GoogleNews-vectors-negative300.bin.gz", binary = True)

In [70]:
def sentencVector(model, sentence):
    sen = str(sentence).lower()
    words = word_tokenize(sen)
    #words = [w for w in words if w not in stop_words]
    words = [w for w in words if len(w) > 1]
    sen_v = []
    for word in words:
        try:
            sen_v.append(model[word])
        except:
            continue
    sen_v = np.array(sen_v)
    sen_v = np.sum(sen_v, axis = 0)
    return sen_v  

In [65]:
WORD_VEC_DIM = 300

def batch_sent_vec(model, sentences):
    rows = len(sentences)
    sents_matrix = np.zeros((rows, WORD_VEC_DIM))
    for i in range(rows):
        sents_matrix[i,:] = sentencVector(model, sentences[i])
    return sents_matrix

In [12]:
#分组数据
from keras.utils import np_utils

def grouper(iterable, n):
    args = [iter(iterable)] * n
    return zip(*args)
# 处理Y（回答）
from sklearn import preprocessing

def deal_all_answer(answers):
    labelEncoder = preprocessing.LabelEncoder()
    labelEncoder.fit(answers)
    return labelEncoder

def get_answer_matrix(answers, encoder):
    y = encoder.transform(answers)
    Y = np_utils.to_categorical(y, encoder.classes_.shape[0])
    return Y

In [6]:
#将图片的特征数据与句子的vector一起，作为输入数据传给MLP网络
#读取数据：
questions = open("./questions_train2014.txt", "r").read().splitlines()
img_ids = open("./images_train2014.txt", "r").read().splitlines()
answers = open("./answers_train2014_modal.txt","r").read().splitlines()
#准备图片特征数据
img_feature_struct, img_id_feature_map = generate_coco_image_feature("./vgg_feats.mat", "./coco_vgg_IDMap.txt")

index = (int)(len(questions) * 0.8)
train_questions=questions[0:index]
train_img_ids = img_ids[0:index]
train_answers = answers[0:index]

test_questions = questions[index:]
test_img_ids = img_ids[index:]
test_answers = answers[index:]

#准备训练数据，从answers中找出频繁出现的1000个回答，再根据这些回答，找出train数据中的问题，图片，测试数据中的问题，图片
from collections import defaultdict

#1. 找出最频繁出现的1000个回答
max_ans = 1000
answers_all = defaultdict(int)
for ans in train_answers:
    answers_all[ans] += 1
answers_all = sorted(answers_all.items(), key=operator.itemgetter(1), reverse= True)[0:max_ans]
top_ans, top_freq = zip(*answers_all)
q_new , img_new, ans_new = [], [], []

#2.找出train数据中的问题，图片
for q, img, ans in zip(train_questions, train_img_ids, train_answers):
    if ans in top_ans:
        q_new.append(q)
        img_new.append(img)
        ans_new.append(ans)

train_questions_mlp  = q_new
train_img_ids_mlp = img_new
train_answers_mlp = ans_new

#处理回答
answers_encoder = deal_all_answer(train_answers_mlp)

In [72]:
#MLP模型相关参数
num_hidden_units = 1024
num_hidden_layers = 3
dropout = 0.5
activation_fun = 'relu'
# 图片的维度大小
img_dim = 4096

model_save_interval = 10
batch_size = 128

#建立模型
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD

model = Sequential()
model.add(Dense(num_hidden_units, input_dim = img_dim + WORD_VEC_DIM, kernel_initializer='uniform'))
model.add(Activation(activation_fun))
model.add(Dropout(dropout))

#中间层
for i in range(num_hidden_layers-1):
    model.add(Dense(num_hidden_units, kernel_initializer='uniform'))
    model.add(Activation(activation_fun))
    model.add(Dropout(dropout))

#输出层
model.add(Dense(max_ans, kernel_initializer='uniform'))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

In [8]:
#保存模型图
#显示模型图片
from keras.utils import plot_model
from IPython.display import Image

def plot_and_show_model(model, file):
    plot_model(model, to_file=file, show_shapes= True)
    Image(filename=file)

In [74]:
#train
from random import shuffle
from keras.utils import generic_utils

#只是看下效果
num_epochs = 10

for i in range(num_epochs):
    #准备数据
    # 打乱数据的顺序
    index_shuffle = [i for i in range(len(train_questions_mlp))]
    shuffle(index_shuffle)
    
    train_questions_mlp = [train_questions_mlp[i] for i in index_shuffle]
    train_img_ids_mlp = [train_img_ids_mlp[i] for i in index_shuffle]
    train_answers_mlp = [train_answers_mlp[i] for i in index_shuffle]
    
    progbar = generic_utils.Progbar(len(train_questions_mlp))
    
    for ques_batch, img_batch, ans_batch in zip(grouper(train_questions_mlp, batch_size), grouper(train_img_ids_mlp, batch_size), grouper(train_answers_mlp, batch_size)):
        
        X_questions = batch_sent_vec(w2v_model, ques_batch)
        X_img = get_image_matrix(img_feature_struct, img_id_feature_map, img_batch)
        X = np.hstack((X_questions, X_img))
        Y = get_answer_matrix(ans_batch, answers_encoder)
        
        loss = model.train_on_batch(X, Y)
        progbar.add(batch_size, values=[("train loss", loss)])
        #保存中间训练的模型
        if i % model_save_interval == 0 :
            model.save_weights("vqa_mlp_model_epoch{:02d}.hdf5".format(i))
    
model.save_weights("vqa_mlp_model_epoch{:02d}.hdf5".format(i))

172416/172471 [============================>.] - ETA: 0s - train loss: 4.1063

In [119]:
def get_question_maxtrix_for_rnn(questions, timestep):
    rows = len(questions)
    q_mat = np.zeros((rows, timestep, WORD_VEC_DIM))
    for i in range(rows):
        words = word_tokenize(questions[i])
        words = [word for word in words if len(word) > 1]
        for j in range(len(words)):
            if j < timestep:
                try:
                    M = w2v_model[words[j]]
                    q_mat[i,j,:] = M
                except:
                    continue
    return q_mat

In [125]:
#建立模型：LSTM处理question, 然后再使用MLP
# 参数们
from keras.layers.core import Reshape
from keras.layers import Concatenate
from keras.layers.recurrent import LSTM
from keras.layers import Input
from keras import Model

max_qu_len = 30

num_hidden_units_mlp = 1024
num_hidden_units_lstm = 512
num_hidden_layers_mlp = 3

#建立一层或三层LSTM
def vqa_lstm_mlp(is_one_layer,model_file):
    img_input = Input(shape=(img_dim,))
    img_model = Reshape((img_dim,), input_shape=(img_dim,))(img_input)

    #语言模型
    lan_input = Input(shape=(max_qu_len, WORD_VEC_DIM))
    if is_one_layer:
        lan_model = LSTM(units = num_hidden_units_lstm, return_sequences = False, input_shape=(max_qu_len, WORD_VEC_DIM))(lan_input)
    else:
        #输入层
        lan_m1 = LSTM(units = num_hidden_units_lstm, return_sequences = True, input_shape=(max_qu_len, WORD_VEC_DIM))(lan_input)
        lan_m2 = LSTM(units = num_hidden_units_lstm, return_sequences = True)(lan_m1)
        lan_model = LSTM(units = num_hidden_units_lstm, return_sequences = False)(lan_m2)

    concate_model = Concatenate(axis = 1)([lan_model, img_model])

    hid_1 = Dense(num_hidden_units_mlp, init='uniform', activation=activation_fun)(concate_model)
    drop_out_1 = Dropout(dropout)(hid_1)
    hid_2 = Dense(num_hidden_units_mlp, init='uniform', activation=activation_fun)(drop_out_1)
    drop_out_2 = Dropout(dropout)(hid_2)
    hid_3 = Dense(num_hidden_units_mlp, init='uniform', activation=activation_fun)(drop_out_2)
    drop_out_3 = Dropout(dropout)(hid_3)

    out = Dense(max_ans, activation= 'softmax')(drop_out_3)

    merge_mlp_model = Model(inputs=[lan_input, img_input], outputs= out)
    #保存model
    json_str = merge_mlp_model.to_json()
    open(model_file,"w").write(json_str)

    merge_mlp_model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
    return merge_mlp_model

merge_mlp_model = vqa_lstm_mlp(True, "./vqa_lstm_mlp_model.json")
    

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:32: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1024, activation="relu", kernel_initializer="uniform")`
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:34: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1024, activation="relu", kernel_initializer="uniform")`
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:36: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1024, activation="relu", kernel_initializer="uniform")`


In [122]:
#train
def train_model(model, num_epochs, get_qus_matrix_fun, model_weight_file_name, timesteps, train_questions_mlp, train_img_ids_mlp, train_answers_mlp):
    for i in range(num_epochs):
        progbar = generic_utils.Progbar(len(train_questions_mlp))

        for ques_batch, img_batch, ans_batch in zip(grouper(train_questions_mlp, batch_size), grouper(train_img_ids_mlp, batch_size), grouper(train_answers_mlp, batch_size)):

            X_questions = get_qus_matrix_fun(ques_batch, timesteps)
            X_img = get_image_matrix(img_feature_struct, img_id_feature_map, img_batch)
            X = [X_questions, X_img]
            Y = get_answer_matrix(ans_batch, answers_encoder)

            loss = model.train_on_batch(X, Y)
            progbar.add(batch_size, values=[("train loss", loss)])
            #保存中间训练的模型
            if i % model_save_interval == 0 :
                model.save_weights(model_weight_file_name + "_epoch{:02d}.hdf5".format(i))
    
    model.save_weights(model_weight_file_name + "_epoch{:02d}.hdf5".format(i))

In [123]:
train_model(merge_mlp_model, 1, get_question_maxtrix_for_rnn, "vqa_lstm_mlp_model_weight", max_qu_len, train_questions_mlp, train_img_ids_mlp, train_answers_mlp)

172416/172471 [============================>.] - ETA: 0s - train loss: 3.9917

In [126]:
#可以再尝试下seq2seq (encode-decode,生成模型)
# 加attention
#bert, abert
#1. 多层LSTM：3层
lstm_3_model = vqa_lstm_mlp(False, "./vqa_3_lstm_mlp_model.json")
train_model(lstm_3_model, 1, get_question_maxtrix_for_rnn, "vqa_3_lstm_mlp_model_weight", max_qu_len, train_questions_mlp, train_img_ids_mlp, train_answers_mlp)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:32: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1024, activation="relu", kernel_initializer="uniform")`
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:34: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1024, activation="relu", kernel_initializer="uniform")`
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:36: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1024, activation="relu", kernel_initializer="uniform")`


172416/172471 [============================>.] - ETA: 1s - train loss: 3.8341

In [131]:
#双向LSTM
from keras.layers import Bidirectional

def create_bidirect_lstm(one_layer, file_path, timestep, n_class):
    #图片特征输入
    img_input = Input(shape=(img_dim,))
    img_model = Reshape((img_dim,), input_shape=(img_dim,))(img_input)
    
    #语言输入：
    lan_input = Input(shape=(timestep, WORD_VEC_DIM))
    if one_layer:
        lan_model = Bidirectional(LSTM(units = num_hidden_units_lstm, return_sequences = False, input_shape=(timestep, WORD_VEC_DIM)))(lan_input)
    else:
        lan_1 = Bidirectional(LSTM(units = num_hidden_units_lstm, return_sequences = True, input_shape=(timestep, WORD_VEC_DIM)))(lan_input)
        lan_model = Bidirectional(LSTM(units = num_hidden_units_lstm, return_sequences = False))(lan_1)
    
    #合并图片与语言特征
    concate = Concatenate(axis=1)([lan_model, img_model])
    
    #MLP层 搭三层
    lay1 = Dense(num_hidden_units_mlp, init='uniform',activation=activation_fun)(concate)
    drop1 = Dropout(dropout)(lay1)
    lay2 = Dense(num_hidden_units_mlp, init='uniform',activation=activation_fun)(drop1)
    drop2 = Dropout(dropout)(lay2)
    lay3 = Dense(num_hidden_units_mlp, init='uniform',activation=activation_fun)(drop2)
    drop3 = Dropout(dropout)(lay3)
    out = Dense(n_class, activation= 'softmax')(drop3)
    
    model = Model(inputs=[lan_input, img_input], outputs = out)
     #保存model
    json_str = model.to_json()
    open(file_path,"w").write(json_str)
    
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
    
    return model
    

In [132]:
timestep = 25
model_bi_lstm = create_bidirect_lstm(True, "./vqa_bi_lstm_mlp_model.json", timestep, max_ans)
model_2_bi_lstm = create_bidirect_lstm(True, "./vqa_bi_lstm_mlp_model.json", timestep, max_ans)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1024, activation="relu", kernel_initializer="uniform")`
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1024, activation="relu", kernel_initializer="uniform")`
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1024, activation="relu", kernel_initializer="uniform")`


In [133]:
#train one bidrection lstm
train_model(model_bi_lstm, 1, get_question_maxtrix_for_rnn, "vqa_bi_lstm_mlp_model_weight", timestep, train_questions_mlp, train_img_ids_mlp, train_answers_mlp)

172416/172471 [============================>.] - ETA: 0s - train loss: 3.1875

In [134]:
#train 2 bidrection lstm
train_model(model_2_bi_lstm, 1, get_question_maxtrix_for_rnn, "vqa_2_bi_lstm_mlp_model_weight", timestep, train_questions_mlp, train_img_ids_mlp, train_answers_mlp)

172416/172471 [============================>.] - ETA: 0s - train loss: 3.2127